In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

from ISLP import load_data
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer

# Linear Regression

## Simple Linear Regression

In [ ]:
boston = load_data("Boston")
boston.columns

In [ ]:
# Fit OLS model
X = pd.DataFrame(
    {
        "intercept": np.ones(boston.shape[0]),
        "lstat": np.array(boston["lstat"]),
    }
)
y = np.array(boston["medv"])

model = sm.OLS(endog=y, exog=X)
results = model.fit()

In [ ]:
results.summary()

In [ ]:
# Predict
X_test = pd.DataFrame(
    {
        "intercept": np.array([1., 1., 1.]),
        "lstat": np.array([5., 10., 15.])
    }
)
y_hat = results.get_prediction(X_test)

In [ ]:
y_hat.predicted_mean

In [ ]:
# Confidence intervals
y_hat.conf_int(alpha=0.05)

In [ ]:
# Prediction intervals
y_hat.conf_int(obs=True, alpha=0.05)

In [ ]:
# Plot

def abline(ax, b, m, *args, **kwargs):
    "Add a line with slope m and intercept b to ax"
    xlim = ax.get_xlim()
    ylim = [m * xlim[0] + b, m * xlim[1] + b]
    ax.plot(xlim, ylim, *args, **kwargs)


ax = boston.plot.scatter(x="lstat", y="medv")
abline(
    ax,
    results.params[0],
    results.params[1],
    "r--",
    lw=3
)

In [ ]:
# Residual analysis
fig, ax = plt.subplots()
ax.scatter(results.fittedvalues, results.resid)
ax.set(xlabel="Fitted value", ylabel="Residual")
ax.axhline(0, c="k", ls="--");

In [ ]:
infl = results.get_influence()
fig, ax = plt.subplots()
ax.scatter(np.arange(X.shape[0]), infl.hat_matrix_diag)
ax.set(xlabel="Index", ylabel="Levarge")

np.argmax(infl.hat_matrix_diag)

## Multiple Linear Regression

In [ ]:
X = pd.DataFrame(
    {
        "intercept": np.ones(boston.shape[0]),
        "lstat": np.array(boston["lstat"]),
        "age": np.array(boston["age"]),
    }
)
y = np.array(boston["medv"])

model = sm.OLS(endog=y, exog=X)
results = model.fit()
results.summary()

In [ ]:
# Backward selection based on p-values
X = boston.drop(columns=["medv", "indus", "age"])
X.insert(loc=0, column="intercept", value=np.ones(X.shape[0]))
y = np.array(boston["medv"])
model = sm.OLS(endog=y, exog=X)
results = model.fit()
results.summary()

In [ ]:
# Collinearity
vals = [VIF(X, i) for i in range(1, X.shape[1])]
vif = pd.DataFrame({"vif": vals}, index=X.columns[1:])
vif

## Non-Linear and Interaction Transforms

In [ ]:
X = boston.drop(columns=["medv", "indus"])
X.insert(loc=0, column="intercept", value=np.ones(X.shape[0]))
y = np.array(boston["medv"])
model = sm.OLS(endog=y, exog=X)
results1 = model.fit()
results.summary()

In [ ]:
X = boston.drop(columns=["medv", "indus"])
X.insert(loc=0, column="intercept", value=np.ones(X.shape[0]))
y = np.array(boston["medv"])

# Feature transforms
poly_features = ["lstat", "age"]
poly_transformer = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
col_transformer = ColumnTransformer(
    transformers=[("poly", poly_transformer, poly_features),],
    remainder="passthrough",
    verbose_feature_names_out=False,
)

X = col_transformer.fit_transform(X, y)
X = pd.DataFrame(X, columns=col_transformer.get_feature_names_out())

model = sm.OLS(endog=y, exog=X)
results2 = model.fit()
results.summary()

In [ ]:
anova_lm(results1, results2)

In [ ]:
# Residual analysis
fig, ax = plt.subplots()
ax.scatter(results2.fittedvalues, results2.resid)
ax.set(xlabel="Fitted value", ylabel="Residual")
ax.axhline(0, c="k", ls="--");